<a href="https://colab.research.google.com/github/saishshinde15/CrewAI/blob/main/Using_CrewAI_with_langchain_Tools_of_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install crewai crewai_tools langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install -qU langchain_community pypdf

In [3]:
%pip install -qU langchain-text-splitters

In [4]:
pip install -qU langchain-cohere

In [5]:
pip install -qU "langchain-chroma>=0.1.2"

In [6]:
%pip install -qU "langchain-community>=0.2.11" tavily-python

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [8]:
loader=PyPDFLoader('/content/HAI_AI-Index-Report-2024.pdf')

In [9]:
docs=loader.load()

In [10]:
from langchain_text_splitters import CharacterTextSplitter

In [11]:
text_splitter=CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs=text_splitter.split_documents(docs)
docs[4]

Document(metadata={'source': '/content/HAI_AI-Index-Report-2024.pdf', 'page': 4}, page_content='Artificial Intelligence\nIndex Report 2024\n5\nTop 10 Takeaways\n1. AI beats humans on some tasks, but not on all. AI has surpassed human performance on several \nbenchmarks, including some in image classification, visual reasoning, and English understanding. Yet it trails \nbehind on more complex tasks like competition-level mathematics, visual commonsense reasoning and planning.\n2. Industry continues to dominate frontier AI research.  In 2023, industry produced 51 notable \nmachine learning models, while academia contributed only 15. There were also 21 notable models resulting from \nindustry-academia collaborations in 2023, a new high.\n3. Frontier models get way more expensive. According to AI Index estimates, the training costs \nof state-of-the-art AI models have reached unprecedented levels. For example, OpenAI’s GPT-4 used an \nestimated $78 million worth of compute to train, while 

In [12]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

In [13]:
import getpass

inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")


Enter your HF Inference API Key:

··········


In [14]:

embeddings = HuggingFaceInferenceAPIEmbeddings(model_name="sentence-transformers/all-MiniLM-l6-v2",api_key=inference_api_key)

In [15]:
from langchain_chroma import Chroma

In [16]:
db=Chroma.from_documents(docs,embeddings,persist_directory='./db',collection_name='cusotom_tool')

In [17]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

llm = ChatCohere()

··········


In [18]:
from langchain.chains import RetrievalQA

In [19]:
qa_tool=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
)

In [20]:
qa_tool.run("What is the main topic of the report?")

<ipython-input-20-fcad05f29161>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_tool.run("What is the main topic of the report?")


'The main topic of the report is Artificial Intelligence. It is titled "Artificial Intelligence Index Report 2024" and covers various aspects of AI, including research and development, technical performance, responsible AI, economy, science and medicine, education, policy and governance, diversity, and public opinion.'

In [21]:
@tool("AI Index Report 2024 QA Tool")
def qa_tool(question: str) -> str:
    """
    This tool answers questions about the 2024 AI Index Report.
    Provide a question related to the report's content, and it will return the answer based on the report's information.
    """

    try:
      qa = RetrievalQA.from_chain_type(
          llm=llm,
          chain_type="stuff",
          retriever=db.as_retriever(),
      )
      answer = qa.run(question)
      return answer
    except Exception as e:
      return f"An error occurred: {e}"

In [39]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

Tavily API key:
··········


In [40]:
from langchain_community.tools import TavilySearchResults
search_tool=TavilySearchResults(max_results=3)

In [41]:
@tool("Tavily Search Tool")
def search_tool(query: str) -> str:
    """
    Searches the web using Tavily and returns the top results.
    Input should be a search query.
    """
    try:
        results = search_tool.run(query)
        return results
    except Exception as e:
        return f"An error occurred: {e}"

In [22]:
from crewai import Agent, Task, Crew

In [58]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
  tools=[qa_tool],
  llm=llm,

)

In [59]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
  #tools=[search_tool], using the tool in the task as the all the agents are using the tools
  llm=llm
)

In [60]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
    agent=support_agent,
)

In [61]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."

    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
    #tools=[search_tool], no need of search tool(not getting used to full potential as there is no need)
)


In [62]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=False,
  #embedder={  #Set memory as True when using this line of code
        #"provider": "cohere",
        #"config": {
           # "api_key": "Qt91rv5SJI7AmM732oGx9CsHRVN6CuOX5Wb8XtKS",
            #"model_name": "<model_name>"
       # }
    #}
)

In [64]:
inputs = {
    "customer": "Tethys Ai",
    "person": "Saish Shinde",
    "inquiry": "I need help with reviewing the pdf i.e what is the AI Index Report 2024 about?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Tethys Ai just reached out with a super important ask:
I need help with reviewing the pdf i.e what is the AI Index Report 2024 about?

Saish Shinde from Tethys Ai is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Using tool: AI Index Report 2024 QA Tool
## Tool Input: 
"{\"question\": \"What is the main topic of the AI Index Report 2024?\"}"
## Tool Output: 
The AI Index Report 2024 is an annual report that provides a comprehensive overview of the state of artificial intelligence (AI) and its impact on various sectors and industries. It covers a wide range of topics related to AI, including research, development, applications, ethics, policy, and more. The report aims to track and measure the progress, growth, and trends in the field of AI, offering insights

In [65]:
print(result)

# AI Index Report 2024: Unlocking Insights into the AI Landscape

## Introduction

Saish Shinde, we appreciate your interest in the AI Index Report 2024! This report is an invaluable resource for anyone seeking a comprehensive understanding of the artificial intelligence landscape. Let's dive into the details and provide a thorough response to your inquiry.

## Overview of the AI Index Report 2024

The AI Index Report 2024 is an annual publication that serves as a comprehensive guide to the diverse world of AI. It aims to provide a 360-degree view of global AI activities, covering various aspects such as research, development, applications, ethics, and policy. The report is designed to be accessible and informative for a wide range of audiences, from researchers and industry experts to policymakers and enthusiasts.

## Key Focus Areas

### Global AI Activity Assessment
- **Research Output:** The report analyzes the volume and quality of AI research publications, tracking the growth and